# Analyzing Hubway Trips in SQL

In [189]:
#You will have to install ipython-sql using the following command:

!pip install ipython-sql --upgrade

#Just Execute the cell and it will install

Defaulting to user installation because normal site-packages is not writeable
  Using cached SQLAlchemy-2.0.21-cp310-cp310-win_amd64.whl (2.0 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.49
    Uninstalling SQLAlchemy-1.4.49:
      Successfully uninstalled SQLAlchemy-1.4.49


In [190]:
# If your file is in another Directory (Without Space in Name of Directory) : %sql sqlite:///Hello/Another_Folder/hubway_small_ok.db

# If your file is in Same Directory : %sql sqlite:///hubway_small_ok.db

%load_ext sql
%sql sqlite:///hubway.db
%config SqlMagic.autocommit=False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


**Please write your queries below in the empty boxes same as you write in a SQL Window**

Just append **%sql** before your query like this '%sql Select * from Trips limit 5;'

In [191]:
%%sql

Select * from Trips limit 2

 * sqlite:///hubway.db
Done.


id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male


**About the Dataset and Schema**

The database has two tables, trips and stations. To begin with, we'll look at the trips table. It contains the following columns:


**Trips:**

id — A unique integer that serves as a reference for each trip

duration — The duration of the trip, measured in seconds

start_date — The date and time the trip began

start_station — An integer that corresponds to the id column in the stations table for the station the trip started at

end_date — The date and time the trip ended

end_station — The 'id' of the station the trip ended at

bike_number — Hubway's unique identifier for the bike used on the trip

sub_type — The subscription type of the user. "Registered" for users with a membership, "Casual" for users without a membership

zip_code — The zip code of the user (only available for registered members)

birth_date — The birth year of the user (only available for registered members)

gender — The gender of the user (only available for registered members)


**Station:**


id — A unique identifier for each station (corresponds to the start_station and end_station columns in the trips table)

station — The station name

municipality — The municipality that the station is in (Boston, Brookline, Cambridge or Somerville)

lat — The latitude of the station

lng — The longitude of the station

## Analytical Questions Test in SQL

Q1. What was the duration of the longest trip? What was the average trip duration? What was the smallest trip duration?

In [192]:
%%sql
Select min(duration) smallest_trip ,max(duration) longest_trip ,avg(duration) Average_trip  from Trips;

 * sqlite:///hubway.db
Done.


smallest_trip,longest_trip,Average_trip
0,9999,912.4096819046612


Q2. How many trips were taken by 'Registered' users?

Hint: Use Sub Type to Filter by Where

In [193]:
%%sql
SELECT count(distinct id) as count
FROM Trips
WHERE sub_type = 'Registered';


 * sqlite:///hubway.db
Done.


count
1105192


#### 1105192 trips were taken by Registered users.

Q3. How many trips were taken by male users in comparison to Female users? for Registered users only

Hint: Group By Gender and Calculate Count

In [194]:
%%sql
select gender , count(distinct id)as count
from trips
where sub_type = 'Registered'
group by gender;


 * sqlite:///hubway.db
Done.


gender,count
Female,271333
Female,1
Male,833858


To clean female column we run another querry to get clean results.

In [195]:
%%sql

select (case when gender like 'Female%' then 'Female' else gender end) as cleaned_gender
, count(distinct id) as no_of_trips
from  trips
where sub_type = 'Registered'
group by cleaned_gender 
order by no_of_trips desc;

 * sqlite:///hubway.db
Done.


cleaned_gender,no_of_trips
Male,833858
Female,271334


#### Male took more rides in comparision to Female

Q4. Do registered or casual users take longer trips?

Hint: Group By Sub Type and Calculate Average

In [196]:
%%sql

select sub_type ,  avg(duration) Average_trip 
from trips 
group by sub_type
order by Average_trip desc;

 * sqlite:///hubway.db
Done.


sub_type,Average_trip
Casual,1519.6438967403815
Registered,657.0260669639302


#### Casual users took longer trips than registered users.

Q5. Which bike was used for the most trips?

Hint : Use Bike Number and Limit Results to 1

In [197]:
%%sql

SELECT bike_number , count(distinct id)as count
from trips
group by bike_number
order by count desc
limit 1;


 * sqlite:///hubway.db
Done.


bike_number,count
B00490,2120


#### B00490 is the bike which used for most trips.

Q6. What is the average duration of trips by users over the age of 30?

Hint : DoB is already in years and use Airthmetic operator to caluculate Age like (2021-DOB)

Link : #GETTING AGE IN SQLLITE : https://stackoverflow.com/questions/49929953/how-to-calculate-approximate-age-based-on-birth-year

In [198]:
%%sql

SELECT  avg(duration) as avg_duration 
from trips
where (strftime('%Y', date('now')) - birth_date) >30;


 * sqlite:///hubway.db
Done.


avg_duration
658.4913889465072


#### Average duration of trips over the age of 30 is 658.491(seconds).

Q7. Which stations are most frequently used for round trips?

Hint: Round trip is considered as where Start and Stop Stations are same. Limit to top 5 Stations

In [202]:
%%sql

select s.station , count(distinct t.id) no_round_trips
from trips t
left join stations s on t.start_station = s.id
where t.start_station = t.end_station
group by station
order by no_round_trips desc
limit 5;

 * sqlite:///hubway.db
Done.


station,no_round_trips
The Esplanade - Beacon St. at Arlington St.,3064
Charles Circle - Charles St. at Cambridge St.,2739
Boston Public Library - 700 Boylston St.,2548
Boylston St. at Arlington St.,2163
Beacon St / Mass Ave,2144


Q8. How many trips start and end in different municipalities?

Hint: We need to JOIN the trips table to the stations table twice. Once ON the start_station column and then ON the end_station

In [203]:
%%sql

SELECT count(distinct t.id)as diff_municipalities_rides 
FROM trips t
LEFT JOIN stations ss ON t.start_station = ss.id
LEFT JOIN stations es ON t.end_station = es.id
where ss.municipality <> es.municipality;

 * sqlite:///hubway.db
Done.


diff_municipalities_rides
309748


Q9. How many trips incurred additional fees (lasted longer than 30 minutes)?

Hint: Use an Arithmetic function to calculate Duration in Minutes and Use in Where Clause

In [204]:
%%sql

select count(distinct id)as count_trips
from trips
where duration > 60*30;

 * sqlite:///hubway.db
Done.


count_trips
123155


Q10. Which bike was used for the longest total time? Provide the answer in Hours

Hint: Use an Group by Bike Number to sum Duration in Seconds and Limit Result to 1

In [205]:
%%sql

select bike_number , sum(duration)/3600 as total_usage_hours
from trips
group by bike_number
order by total_usage_hours desc
limit 1;

 * sqlite:///hubway.db
Done.


bike_number,total_usage_hours
B00490,571


#### Bike B00490 was used for the longest total time , 571 hours.

Q11. Did registered or casual users take more round trips?

Hint: Recall Q7 and Modify your query to answer by Sub Type

In [206]:
%%sql

select sub_type,  count(distinct id) as count_round_trips
from trips
where start_station = end_station
group by sub_type
order by count_round_trips desc;

 * sqlite:///hubway.db
Done.


sub_type,count_round_trips
Casual,41427
Registered,31635


#### Casual users = 41427
#### Registered users = 31635

#### Casual users took more round trips as compared to Registered users.

Q12. Which municipality had the most frequent Station End?

Hint: Join the Station table on End Station and then Group by Municipality to find answer

In [207]:
%%sql

SELECT s.municipality, COUNT(t.id) AS trip_count
FROM trips t
JOIN stations s ON t.end_station = s.id
GROUP BY s.municipality
ORDER BY trip_count DESC
LIMIT 1;



 * sqlite:///hubway.db
Done.


municipality,trip_count
Boston,1212364


Q13. Which From and To Routes are Frequent with Regards to Municipalities?

Hint: Get Start and End Municipalities and Group by Both Attributes for Count of Rides

In [208]:
%%sql

SELECT s_start.municipality AS start_municipality,  
s_end.municipality AS end_municipality, 
COUNT(t.id) AS trip_count
FROM trips t
JOIN stations s_start ON t.start_station = s_start.id
JOIN stations s_end ON t.end_station = s_end.id
GROUP BY s_start.municipality, s_end.municipality
ORDER BY trip_count DESC
LIMIT 1;

 * sqlite:///hubway.db
Done.


start_municipality,end_municipality,trip_count
Boston,Boston,1081805


Q14. Which Age Band in "Registered Users" has the most Rides? Calculate Age Bands based on interval of 10 and Cut Off by 70 or Above

Hint: Use Case When to Create Range of Value Bands as per Above Criteria then Group By. Use DoB Years to Calculate Age First (https://youtu.be/G46GIIRbO-g?t=76)

In [209]:
%%sql
SELECT
    count(DISTINCT id) count_rides,
    CASE
        WHEN (2023 - birth_date) <= 9 THEN '0-9'
        WHEN (2023 - birth_date) >= 10 AND (2023 - birth_date) < 20 THEN '10-19'
        WHEN (2023 - birth_date) >= 20 AND (2023 - birth_date) < 30 THEN '20-29'
        WHEN (2023 - birth_date) >= 30 AND (2023 - birth_date) < 40 THEN '30-39'
        WHEN (2023 - birth_date) >= 40 AND (2023 - birth_date) < 50 THEN '40-49'
        WHEN (2023 - birth_date) >= 50 AND (2023 - birth_date) < 60 THEN '50-59'
        WHEN (2023 - birth_date) >= 60 AND (2023 - birth_date) < 70 THEN '60-69'
        WHEN (2023 - birth_date) >= 70 THEN '70+'
    END AS age_bands
FROM trips
WHERE sub_type = 'Registered' AND birth_date IS NOT NULL
GROUP BY age_bands
ORDER BY count_rides DESC
limit 1;

 * sqlite:///hubway.db
Done.


count_rides,age_bands
119882,40-49


# Analyzing Hubway Trips in Python

In [210]:
#You will have to install SQLLite Connector using the following command:

!pip install sqlite3

#Just Execute the cell and it will install

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [211]:
# MANDATORY CELL TO EXECUTE FIRST SO WE HAVE DATA IN THE RIGHT TYPE

import sqlite3, pandas as pd, numpy as np

cnx = sqlite3.connect('hubway.db')

cnx.cursor().executescript("""

UPDATE trips SET duration       = NULL WHERE duration IS '';
UPDATE trips SET start_date     = NULL WHERE start_date IS '';
UPDATE trips SET start_station  = NULL WHERE start_station IS '';
UPDATE trips SET end_date       = NULL WHERE end_date IS '';
UPDATE trips SET end_station    = NULL WHERE end_station IS '';
UPDATE trips SET sub_type       = NULL WHERE sub_type IS '';
UPDATE trips SET zip_code       = NULL WHERE zip_code IS '';
UPDATE trips SET birth_date     = NULL WHERE birth_date IS '';
UPDATE trips SET gender         = NULL WHERE gender IS '';

""")

trips=pd.read_sql_query("SELECT * FROM trips", cnx)
stations=pd.read_sql_query("SELECT * FROM stations", cnx)

print(len(trips))
print(len(stations))

trips=trips\
    .replace(r'^\s*$', np.nan, regex=True)\
    .astype({\
            'start_date':'datetime64'\
            ,'end_date':'datetime64'\
            ,'start_station':pd.Int64Dtype()\
            ,'end_station':pd.Int64Dtype()\
            ,'birth_date':pd.Int64Dtype()\
            })

stations=stations\
    .replace(r'^\s*$', np.nan, regex=True)\
    .astype({\
            'lat':'float64'\
            ,'lng':'float64'\
            })

print(len(trips))
print(len(stations))

cnx.close()

1570001
142
1570001
142


In [212]:
# CHECKING IF THE DATA IS LOAD INTO RIGHT TABLES IN PYTHON DF

print(trips.head(5))

print('\n ------- \n')

print(stations.head(5))

   id  duration          start_date  start_station            end_date  \
0   1         9 2011-07-28 10:12:00             23 2011-07-28 10:12:00   
1   2       220 2011-07-28 10:21:00             23 2011-07-28 10:25:00   
2   3        56 2011-07-28 10:33:00             23 2011-07-28 10:34:00   
3   4        64 2011-07-28 10:35:00             23 2011-07-28 10:36:00   
4   5        12 2011-07-28 10:37:00             23 2011-07-28 10:37:00   

   end_station bike_number    sub_type zip_code  birth_date  gender  
0           23      B00468  Registered   '97217        1976    Male  
1           23      B00554  Registered   '02215        1966    Male  
2           23      B00456  Registered   '02108        1943    Male  
3           23      B00554  Registered   '02116        1981  Female  
4           23      B00554  Registered   '97214        1983  Female  

 ------- 

   id                             station municipality        lat        lng
0   3              Colleges of the Fenway     

In [213]:
# EXAMPLE SOLUTION, REMEMBER YOUR ANSWER SHOULD BE PRECISE AND ON POINT (SORTED AND IN RIGHT FORMAT)

##PYTHON SOLUTION

trips.agg({'duration': ['mean', 'min', 'max']}).T

,mean,min,max
duration,912.409682,0.0,9999.0


In [214]:
trips.head()



,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983,Female


In [215]:
## SOLUTION TO QUESTION 2

##Q2. How many trips were taken by 'Registered' users?

# INSERT CODE HERE

(trips['sub_type'] == 'Registered').sum()

1105192

#### 1105192 trips were taken by Registered users.

In [216]:
## SOLUTION TO QUESTION 3

##Q3. How many trips were taken by male users in comparison to Female users? for Registered users only

# INSERT CODE HERE

print(trips['gender'].unique())  # checking unique values of gender

trips['gender'] = trips['gender'].replace('Female\n' , 'Female')   # cleaning gender values of female

registered = trips[trips['sub_type'] == 'Registered']    # selecting registered users only

registered['gender'].value_counts()       # counts of users



['Male' 'Female' None 'Female\n']


Male      833858
Female    271334
Name: gender, dtype: int64

#### Registered male users took significantly more trips than Registered female users.




In [217]:
## SOLUTION TO QUESTION 4

##Q4. Do registered or casual users take longer trips?
# INSERT CODE HERE

users_duration_mean =  trips.groupby('sub_type')['duration'].mean()

users_duration_mean

sub_type
Casual        1519.643897
Registered     657.026067
Name: duration, dtype: float64

#### Casual users : 1519.64 (seconds)

#### Registered users : 657.02 (seconds)

###  From the above data, it is shown that Casual users take longer trips on average compared to Registered users.


In [218]:
## SOLUTION TO QUESTION 5
##Q5. Which bike was used for the most trips?

# INSERT CODE HERE

bike_most_trips = trips['bike_number'].value_counts().head(1)
bike_most_trips

B00490    2120
Name: bike_number, dtype: int64

#### Bike B00490 has most trips.

In [219]:
## SOLUTION TO QUESTION 6

# INSERT CODE HERE

## Q6. What is the average duration of trips by users over the age of 30?

current_year = pd.Timestamp.now().year

trips['age'] = current_year - trips['birth_date']   # calculating age 

average_duration = trips[trips['age'] > 30]['duration'].mean()

average_duration

658.4913889465072

#### The average duration of trips taken by users over the age of 30 is approximately 658.49 seconds .

In [222]:
## SOLUTION TO QUESTION 7

##Q7. Which stations are most frequently used for round trips?

# INSERT CODE HERE

#round_trips = trips[trips['start_station'] == trips['end_station']]
#round_trips['start_station'].value_counts().head(5)

round_trips = trips[trips['start_station'] == trips['end_station']]

# Perform left join with stations dataframe
merged_data = round_trips.merge(stations, left_on='start_station', right_on='id', how='left')

# Group by station name and count distinct trip IDs
round_trip_counts = merged_data.groupby('station').agg(no_round_trips=('id_x', 'nunique'))

# Step 4: Sort the results by count in descending order
sorted_round_trip_counts = round_trip_counts.sort_values(by='no_round_trips', ascending=False)

sorted_round_trip_counts.head()


,no_round_trips
station,
The Esplanade - Beacon St. at Arlington St.,3064
Charles Circle - Charles St. at Cambridge St.,2739
Boston Public Library - 700 Boylston St.,2548
Boylston St. at Arlington St.,2163
Beacon St / Mass Ave,2144


In [223]:
## SOLUTION TO QUESTION 8

## Q8. How many trips start and end in different municipalities?

# INSERT CODE HERE


In [224]:
  
# chaecking null values of start station tu merge the dataframe with stations
trips.isnull().sum() 

id                     0
duration               0
start_date             0
start_station         13
end_date               0
end_station           28
bike_number          450
sub_type               0
zip_code          464988
birth_date       1219985
gender            464809
age              1219985
dtype: int64

In [225]:
# Calculating the percentage of missing values in the "start_station" column
missing_percentage_start = (trips['start_station'].isna().sum() / len(trips)) * 100
missing_percentage_end = (trips['end_station'].isna().sum() / len(trips)) * 100

print("Null values in start_station : " , trips['start_station'].isna().sum())
print('Percentage of missing values in dataset : '  ,missing_percentage_start)

print("\nNull values in end_station : " , trips['end_station'].isna().sum())
print('Percentage of missing values in dataset : '  ,missing_percentage_end)

Null values in start_station :  13
Percentage of missing values in dataset :  0.0008280249503025795

Null values in end_station :  28
Percentage of missing values in dataset :  0.0017834383544978634


 The missing data percentage is extremely low that's why dropping the missing values from the data set.

In [226]:

# Exclude rows with missing start_station or end_station
filtered_trips = trips.dropna(subset=['start_station', 'end_station'])

In [227]:
# Merge the trips dataset with the stations dataset for start_station
trips_with_start_municipality = pd.merge(filtered_trips, stations[['id', 'municipality']], left_on='start_station', right_on='id', how='left')
trips_with_start_municipality = trips_with_start_municipality.rename(columns={'municipality': 'start_municipality'})

# Merge the trips dataset with the stations dataset for end_station
trips_with_both_municipalities = pd.merge(trips_with_start_municipality, stations[['id', 'municipality']], left_on='end_station', right_on='id', how='left')
trips_with_both_municipalities = trips_with_both_municipalities.rename(columns={'municipality': 'end_municipality'})

# Count the trips where the start and end municipalities are different
different_municipalities_count = trips_with_both_municipalities[trips_with_both_municipalities['start_municipality'] != trips_with_both_municipalities['end_municipality']].shape[0]

different_municipalities_count
 

309748

#### 309748 trips start and end in different municipalities

In [228]:
## SOLUTION TO QUESTION 9

## Q9. How many trips incurred additional fees (lasted longer than 30 minutes)?
# INSERT CODE HERE

duration_in_mins= trips['duration']/60
min_duration_greater_than_30 = duration_in_mins[duration_in_mins > 30]
(min_duration_greater_than_30).count()
#min_duration_greater_than_30

123155

### There are 123155 trips incurred additional fees.

In [229]:
## SOLUTION TO QUESTION 10

##Q10. Which bike was used for the longest total time? Provide the answer in Hours

#Hint: Use an Group by Bike Number to sum Duration in Seconds and Limit Result to 1
# INSERT CODE HERE

trips['duration_hour'] = trips['duration']/3600

most_used_bike = trips.groupby('bike_number')['duration_hour'].sum().sort_values(ascending =  False)
most_used_bike.head(1)

bike_number
B00490    571.845278
Name: duration_hour, dtype: float64

#### Bike number B00490 was used for longest total time that is 571.845 hours approx .

In [230]:
## SOLUTION TO QUESTION 11

## Q11. Did registered or casual users take more round trips?

#Hint: Recall Q7 and Modify your query to answer by Sub Type
# INSERT CODE HERE

#method 1
round_trips = trips[trips['start_station'] == trips['end_station']]
print(round_trips['sub_type'].value_counts())


#method 2
# Group round trips by 'sub_type' and count the occurrences for each type
round_trips_counts = trips[trips['start_station'] == trips['end_station']]
round_trips_counts.groupby('sub_type').size()

Casual        41427
Registered    31635
Name: sub_type, dtype: int64


sub_type
Casual        41427
Registered    31635
dtype: int64

#### Casual users = 41427
#### Registered users = 31635

#### Casual users took more round trips as compared to Registered users.

In [231]:
## SOLUTION TO QUESTION 12


#Q12. Which municipality had the most frequent Station End?

#Hint: Join the Station table on End Station and then Group by Municipality to find answer

# INSERT CODE HERE

filtered_trips = trips[trips['end_station'].notna()]
merged_df = pd.merge(filtered_trips, stations, left_on='end_station', right_on='id')

# Group by municipality and count
merged_df['municipality'].value_counts().head(1)

Boston    1212364
Name: municipality, dtype: int64

In [232]:
## SOLUTION TO QUESTION 13

# Q13. Which From and To Routes are Frequent with Regards to Municipalities?
# INSERT CODE HERE

# Step 1 : Create a dictionary mapping station IDs to their corresponding municipalities using the zip function.
station_to_municipality = dict(zip(stations['id'], stations['municipality']))

# Step 2: Create Routes
trips['start_municipality'] = trips['start_station'].map(station_to_municipality)
trips['end_municipality'] = trips['end_station'].map(station_to_municipality)
trips['route'] = trips[['start_municipality', 'end_municipality']].apply(tuple, axis=1)



#trips.groupby(['start_municipality','end_municipality'])['id'].count().max()   # method 1

# Step 3: Count Routes   # method 2
count_routes = trips['route'].value_counts()
count_routes.head(1)


(Boston, Boston)    1081805
Name: route, dtype: int64

In [233]:
## SOLUTION TO QUESTION 14 (MINI CHALLENGE)

#HELP HINT: https://stackoverflow.com/questions/34165876/trying-to-create-grouped-variable-in-python

# INSERT CODE HERE

##Q14. Which Age Band in "Registered Users" has the most Rides? Calculate Age Bands based on interval of 10 and Cut Off by 70 or Above

## age column is already created in above questions..so moving forward to solve this question

#  bins and labels for age bands with interval of 10 and cutoff at 70 or above

bins = [0, 10, 20, 30, 40, 50, 60, 70, 100]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70+']

# Create age bands
trips['age_bands'] = pd.cut(trips['age'], bins=bins, labels=labels, right=False)

trips['age_bands'].value_counts().sort_index()

trips.groupby('age_bands')['sub_type'].count().sort_values(ascending = False).head(1)

age_bands
40-49    119882
Name: sub_type, dtype: int64

####  The age band "40-49" among "Registered Users" has the most rides.

### BONUS PYTHON QUESTION (10 Marks)

##### Can you show a Report in Python "What are the Top 5 States with Most Number of Rides" For Registered Users Only

##### Hints :

1. Use the Sampled Dataframe for your working, else it would take a lot of time to convert many rows in Original table.

2. You will need Data Type Conversion and Cleaning of Zip Code (Don't fear errors / Learn from them)

3. Where do you get the States? READ THE CELL BELOW

!pip install zipcodes

print('\n')

import zipcodes

##### Gets All the Information Available against ZipCode

print(zipcodes.matching('77429')[0].keys())

['zip_code', 'zip_code_type', 'active', 'city', 'acceptable_cities', 'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes', 'world_region', 'country', 'lat', 'long']

##### Sample Working how to Fetch a Against a ZipCode

zipcodes.matching('77429')[0].get('state')

READ MORE > https://github.com/seanpianka/Zipcodes


In [234]:
!pip install zipcodes

Defaulting to user installation because normal site-packages is not writeable


In [235]:
import zipcodes

In [236]:
#EXECUTE THIS CELL TO CREATE SAMPLED DATAFRAME

registered_trips_sampled=trips.query("sub_type=='Registered'").groupby('zip_code').apply(lambda x: x.sample(10, replace=True)).reset_index(drop = True)

In [237]:
registered_trips_sampled['zip_code'].isna().sum()   # just checking nan values..

0

In [238]:
## YOUR REPORT SOLUTION IN PYTHON

# USE registered_trips_sampled Dataframe and temporarily work on column zip_code by storing in variable 'x'

x=registered_trips_sampled['zip_code']

# CLEAN zip_code COLUMN FROM PUNCTUATIONS
print(x.head())
x= x.str.replace("'" , '')    # cleaned the zipcodes
print('\n--------Cleaned zipcodes--------\n')
print(x.head())

# REPLACE NAN VALUES WITH STRING '0' FOR API TO WORK
x = x.fillna('0')
print(x.shape)

# TRY TO USE LAMBDA FUNCTION IN PYTHON TO GET STATE FOR EACH ROW, ONCE YOUR ARE DONE WITH ABOVE STEPS
get_state = lambda zip_code: zipcodes.matching(zip_code)[0]['state'] if zipcodes.matching(zip_code) else 'N/A'
# TRY TO USE LAMBDA FUNCTION IN PYTHON TO GET STATE FOR EACH ROW, ONCE YOUR ARE DONE WITH ABOVE STEPS

#state_function = (lambda z: zipcodes.matching(z)[0].get('state') )
#registered_trips_sampled['state'] = x.apply(state_function,convert_dtype=False)

# SAVE THE OUTPUT INTO A COLUMN NAMED AS registered_trips_sampled['state']

#registered_trips_sampled['state']=x

0    '00210
1    '00210
2    '00210
3    '00210
4    '00210
Name: zip_code, dtype: object

--------Cleaned zipcodes--------

0    00210
1    00210
2    00210
3    00210
4    00210
Name: zip_code, dtype: object
(5300,)


In [239]:
# SAVE THE OUTPUT INTO A COLUMN NAMED AS registered_trips_sampled['state']
registered_trips_sampled['state'] = x.apply(get_state)


In [240]:
registered_trips_sampled['state'].unique()

array(['N/A', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NJ', 'NY', 'PA', 'DC',
       'VA', 'MD', 'NC', 'GA', 'FL', 'AL', 'OH', 'MI', 'WI', 'MN', 'IL',
       'LA', 'TX', 'CO', 'UT', 'CA', 'OR', 'WA'], dtype=object)

In [241]:
# WE WILL USE THIS DICTIONARY TO MAP STATE NAMES INSTEAD OF ABBREVIATIONS

state_names = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [242]:
# CREATE ANOTHER COLUMN BASED ON STATE DERIVED TO GET STATE NAMES USING ABOVE DICTIONARY USING MAP FUNCTION, SAVE in STATE_NAME

trips['state_name'] = registered_trips_sampled['state'].map(state_names)
trips['state_name'].unique()

array([nan, 'Massachusetts', 'Rhode Island', 'New Hampshire', 'Maine',
       'Vermont', 'Connecticut', 'New Jersey', 'New York', 'Pennsylvania',
       'District of Columbia', 'Virginia', 'Maryland', 'North Carolina',
       'Georgia', 'Florida', 'Alabama', 'Ohio', 'Michigan', 'Wisconsin',
       'Minnesota', 'Illinois', 'Louisiana', 'Texas', 'Colorado', 'Utah',
       'California', 'Oregon', 'Washington'], dtype=object)

In [243]:
# USE THE NEWLY CREATED COLUMN TO MAKE A SUMMARY REPORT FOR EACH STATE NAME BASED ON NUMBER OF TRIPS AND GET TOP 5

top_5_states= trips['state_name'].value_counts().sort_values(ascending= False).head()
top_5_states

Massachusetts    3040
New Hampshire     460
New York          200
Rhode Island      190
Connecticut       160
Name: state_name, dtype: int64

### WE HOPE YOU ENJOYED LEARNING SQL & PYTHON TOGHETHER